# AFL Model - Part 4 - Weekly Predictions
Now that we have explored different algorithms for modelling, we can implement our chosen model and predict this week's AFL games! All you need to do is run the afl_modelling script each Thursday or Friday to predict the following week's games.

In [111]:
# Import Modules
from afl_feature_creation import prepare_afl_features
import afl_data_cleaning
import afl_feature_creation
import afl_modelling
import datetime
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

## Creating The Features For This Weekend's Games
To actually predict this weekend's games, we need to create the same features that we have created in the previous tutorials for the games that will be played this weekend. This includes all the rolling averages, efficiency features, elo features etc. So the majority of this tutorial will be using previously defined functions to create features for the following weekend's games.

### Create Next Week's DataFrame
Let's first get our cleaned afl_data dataset, as well as the odds for next weekend and the 2018 fixture.

In [112]:
# Grab the cleaned AFL dataset and the column order
afl_data = afl_data_cleaning.prepare_afl_data()
ordered_cols = afl_data.columns

# Define a function which grabs the odds for each game for the following weekend
def get_next_week_odds(path):
    # Get next week's odds
    next_week_odds = pd.read_csv(path)
    next_week_odds = next_week_odds.rename(columns={"team_1": "home_team", 
                                                "team_2": "away_team", 
                                                "team_1_odds": "odds", 
                                                "team_2_odds": "odds_away"
                                               })
    return next_week_odds

# Import the fixture
# Define a function which gets the fixture and cleans it up
def get_fixture(path):
    # Get the afl fixture
    fixture = pd.read_csv(path)

    # Replace team names and reformat
    fixture = fixture.replace({'Brisbane Lions': 'Brisbane', 'Footscray': 'Western Bulldogs'})
    fixture['Date'] = pd.to_datetime(fixture['Date']).dt.date.astype(str)
    fixture = fixture.rename(columns={"Home.Team": "home_team", "Away.Team": "away_team"})
    return fixture

next_week_odds = get_next_week_odds("data/weekly_odds.csv")
fixture = get_fixture("data/afl_fixture_2018.csv")

In [113]:
fixture.tail()

,Date,Season,Season.Game,Round,home_team,away_team,Venue
211,2018-08-24,2018,212,NaN,Brisbane,West Coast,Gabba
212,2018-08-24,2018,213,NaN,Carlton,St Kilda,Etihad Stadium
213,2018-08-24,2018,214,NaN,Richmond,Melbourne,MCG
214,2018-08-24,2018,215,NaN,Port Adelaide,Essendon,Adelaide Oval
215,2018-08-24,2018,216,NaN,Fremantle,Collingwood,Optus Stadium


Now that we have these DataFrames, we will define a function which combines the fixture and next week's odds to create a single DataFrame for the games over the next 7 days. To use this function we will need Game IDs for next week. So we will create another function which creates Game IDs by using the Game ID from the last game played and adding 1 to it.

In [114]:
# Define a function which creates game IDs for this week's footy games
def create_next_weeks_game_ids(afl_data):
    odds = get_next_week_odds("data/weekly_odds.csv")

    # Get last week's Game ID
    last_afl_data_game = afl_data['Game'].iloc[-1]

    # Create Game IDs for next week
    game_ids = [(i+1) + last_afl_data_game for i in range(odds.shape[0])]
    return game_ids

# Define a function which creates this week's footy game DataFrame
def get_next_week_df(afl_data):
    # Get the fixture and the odds for next week's footy games
    fixture = get_fixture("data/afl_fixture_2018.csv")
    next_week_odds = get_next_week_odds("data/weekly_odds.csv")
    next_week_odds['Game'] = create_next_weeks_game_ids(afl_data)

    # Get today's date and next week's date and create a DataFrame for next week's games
    todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
    date_in_7_days = (datetime.datetime.today() + datetime.timedelta(days=7)).strftime('%Y-%m-%d')
    fixture = fixture[(fixture['Date'] >= todays_date) & (fixture['Date'] < date_in_7_days)].drop(columns=['Season', 'Season.Game', 'Round'])
    next_week_df = pd.merge(fixture, next_week_odds, on=['home_team', 'away_team'])

    # Split the DataFrame onto two rows for each game
    h_df = next_week_df[['Date', 'Game', 'home_team', 'away_team', 'odds']]
    h_df['Team'] = h_df['home_team']
    h_df['Opposition'] = h_df['away_team']
    h_df['Home?'] = 1
    a_df = next_week_df[['Date', 'Game', 'home_team', 'away_team', 'odds_away']].rename(columns={'odds_away': 'odds'})
    a_df['Team'] = a_df['away_team']
    a_df['Opposition'] = a_df['home_team']
    a_df['Home?'] = 0
    next_week = a_df.append(h_df).sort_values(by='Game')
    return next_week

In [115]:
next_week_df = get_next_week_df(afl_data)
game_ids_next_round = create_next_weeks_game_ids(afl_data)
next_week_df.head()

,Date,Game,home_team,away_team,odds,Team,Opposition,Home?
0,2018-07-20,15345,St Kilda,Richmond,1.14,Richmond,St Kilda,0
0,2018-07-20,15345,St Kilda,Richmond,8.00,St Kilda,Richmond,1
1,2018-07-21,15346,Collingwood,North Melbourne,2.64,North Melbourne,Collingwood,0
1,2018-07-21,15346,Collingwood,North Melbourne,1.59,Collingwood,North Melbourne,1
2,2018-07-21,15347,Sydney,Gold Coast,18.00,Gold Coast,Sydney,0


### Create Each Feature
Now let's append next week's DataFrame to our afl_data DataFrame and then create all the features we used in the [AFL Feature Creation Tutorial](0.2. afl_feature_creation_tutorial.ipynb).

In [116]:
# Append next week's games to our afl_data DataFrame
afl_data = afl_data.append(next_week_df).reset_index(drop=True)
afl_data = afl_data[ordered_cols]

# Create disposal efficiency column
afl_data['disposal_efficiency'] = afl_data['ED'] / afl_data['D']

# Create rolling averages for our statistics
cols_indx_start = afl_data.columns.get_loc("GA")
afl_avgs = afl_feature_creation.create_rolling_averages(afl_data, 6, afl_data.columns[cols_indx_start:])

# Create form between teams feature
afl_avgs = afl_feature_creation.form_between_teams(afl_avgs, 6)

# Apply elos
elos, probs, elo_dict = afl_feature_creation.elo_applier(afl_data, 24)
afl_avgs['home_elo'] = afl_avgs['Game'].map(elos).str[0]
afl_avgs['away_elo'] = afl_avgs['Game'].map(elos).str[1]

# Get elos for next week
afl_avgs.loc[afl_avgs['home_elo'].isna(), 'home_elo'] = afl_avgs[afl_avgs['home_elo'].isna()]['home_team'].map(elo_dict)
afl_avgs.loc[afl_avgs['away_elo'].isna(), 'away_elo'] = afl_avgs[afl_avgs['away_elo'].isna()]['away_team'].map(elo_dict)

# Create Adjusted Margin and then Average it over a 6 game window
afl_avgs = afl_feature_creation.map_elos(afl_avgs)
afl_avgs['Adj_elo_ave_margin'] = afl_avgs['Margin'] * afl_avgs['elo_Opp'] / afl_avgs['elo']
afl_avgs = afl_feature_creation.create_rolling_averages(afl_avgs, 6, ['Adj_elo_ave_margin'])

# Create average elo of opponents beaten/lost to feature
afl_avgs = afl_feature_creation.create_ave_elo_opponent(afl_avgs, 6, beaten_or_lost='beaten')
afl_avgs = afl_feature_creation.create_ave_elo_opponent(afl_avgs, 6, beaten_or_lost='lost')

# Create regular margin rolling average
afl_avgs = afl_feature_creation.create_rolling_averages(afl_avgs, 6, ['Margin'])
afl_avgs.tail()

,Team,odds,Date,home_team,away_team,Behinds,Game,Goals,Home?,Opposition,Opposition Behinds,Opposition Goals,Opposition Points,Points,Round,Venue,Season,Status,GA_ave_6,CP_ave_6,UP_ave_6,ED_ave_6,CM_ave_6,MI5_ave_6,One.Percenters_ave_6,BO_ave_6,K_ave_6,HB_ave_6,D_ave_6,M_ave_6,G_ave_6,B_ave_6,T_ave_6,HO_ave_6,I50_ave_6,CL_ave_6,CG_ave_6,R50_ave_6,FF_ave_6,FA_ave_6,AF_ave_6,SC_ave_6,CCL_ave_6,SCL_ave_6,SI_ave_6,MG_ave_6,TO_ave_6,ITC_ave_6,T5_ave_6,disposal_efficiency_ave_6,form_over_opposition_6,home_elo,away_elo,elo,elo_Opp,Adj_elo_ave_margin_ave_6,average_elo_opponents_beaten_6,average_elo_opponents_lost_6,Margin_ave_6
2973,Carlton,7.00,2018-07-22,Carlton,Hawthorn,NaN,15351,NaN,1,Hawthorn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,126.000000,224.666667,254.500000,9.666667,8.166667,47.000000,6.333333,191.666667,156.833333,348.500000,90.000000,7.333333,7.166667,58.333333,30.833333,41.666667,31.500000,58.333333,38.000000,20.833333,25.000000,1419.833333,1498.666667,10.833333,20.666667,74.666667,4800.333333,64.666667,61.000000,8.000000,0.730546,1,1243.047837,1580.237667,1243.047837,1580.237667,-47.876861,1520.746044,1473.155080,-42.500000
2974,Western Bulldogs,9.40,2018-07-22,West Coast,Western Bulldogs,NaN,15352,NaN,0,West Coast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.500000,140.333333,238.000000,278.166667,9.500000,9.166667,51.666667,7.000000,201.666667,174.500000,376.166667,87.000000,10.833333,6.166667,59.666667,28.666667,51.333333,33.166667,52.333333,36.500000,21.833333,20.000000,1515.333333,1549.500000,11.333333,21.833333,75.666667,5388.500000,74.333333,73.333333,11.166667,0.737464,2,1601.537522,1452.042224,1452.042224,1601.537522,-35.651604,1412.598306,1538.300862,-34.166667
2975,West Coast,1.13,2018-07-22,West Coast,Western Bulldogs,NaN,15352,NaN,1,Western Bulldogs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.333333,137.833333,204.166667,250.000000,12.166667,10.833333,49.500000,4.000000,219.166667,122.333333,341.500000,98.333333,11.833333,8.166667,63.666667,45.333333,50.166667,34.500000,52.666667,37.166667,24.666667,19.833333,1541.500000,1638.500000,12.333333,22.166667,86.666667,5613.833333,70.166667,71.000000,9.666667,0.732286,4,1601.537522,1452.042224,1601.537522,1452.042224,7.284490,1503.657098,1618.250945,7.333333
2976,GWS,2.10,2018-07-22,Port Adelaide,GWS,NaN,15353,NaN,0,Port Adelaide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.166667,150.333333,230.833333,270.500000,10.666667,10.833333,56.333333,5.833333,210.666667,167.666667,378.333333,85.166667,11.166667,11.166667,60.333333,32.333333,54.166667,39.833333,58.166667,37.166667,21.000000,23.000000,1526.666667,1678.166667,11.500000,28.333333,111.333333,5927.666667,71.500000,73.000000,9.666667,0.714490,3,1566.616129,1596.476198,1596.476198,1566.616129,-1.586685,1438.724285,1569.306745,-0.666667
2977,Port Adelaide,1.90,2018-07-22,Port Adelaide,GWS,NaN,15353,NaN,1,GWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.166667,157.833333,218.666667,253.166667,12.166667,12.833333,63.333333,6.166667,221.500000,152.666667,374.166667,87.000000,12.333333,10.500000,77.333333,47.333333,58.166667,41.000000,58.666667,38.000000,23.333333,24.500000,1621.833333,1768.666667,12.333333,28.666667,91.666667,5891.333333,75.000000,75.500000,16.166667,0.675495,2,1566.616129,1596.476198,1566.616129,1596.476198,23.166064,1400.051819,1553.088485,25.833333


Our DataFrame looks great! Although there are some NaNs, these are only in columns like 'Goals' which obviously don't have a value yet as the game hasn't been played.

Now we need to get our DataFrame on one line so that each row corresponds to one footy game. Let's use our previously defined function for this. Let's also drop the columns with NaNs which we don't need.

In [117]:
# Get each footy match on individual rows and drop irrelevant columns
one_line = afl_modelling.get_df_on_one_line(afl_avgs)

# Drop duplicate columns and unnecessary columns
cols_to_drop = ['Opposition Behinds', 'Goals', 'Behinds', 'Opposition Goals', 'Opposition Points', 'Points', 'Round', 'Venue', 'Season', 'Status',
               'CCL_ave_6', 'SCL_ave_6', 'SI_ave_6', 'MG_ave_6', 'TO_ave_6', 'ITC_ave_6', 'T5_ave_6', 'elo', 'elo_Opp', 'Behinds_away',
               'Goals_away', 'home_elo_away', 'away_elo_away', 'elo_away', 'elo_Opp_away']
one_line = one_line.drop(columns=cols_to_drop)
# Drop all columns where home_elo or away_elo == 1500 exactly, as this is the first game played
one_line = one_line[(one_line['home_elo'] != 1500) & (one_line['away_elo'] != 1500)]

# Drop Na rows from calculating moving averages
one_line = one_line.dropna(axis=0)

Now let's create our differential_df and then filter out DataFrame to only include this weekend's game, based on the Game IDs we created earlier

In [118]:
diff_df = afl_modelling.get_diff_df(one_line)
diff_df = diff_df.drop(columns=['odds', 'odds_away']).select_dtypes(include=[np.number])
prediction_feature_set = diff_df[diff_df['Game'].isin(game_ids_next_round)].dropna(axis=1)

In [119]:
prediction_feature_set

,Game,home_elo,away_elo,CCL_ave_6_away,SCL_ave_6_away,SI_ave_6_away,MG_ave_6_away,TO_ave_6_away,ITC_ave_6_away,T5_ave_6_away,GA_ave_6_diff,CP_ave_6_diff,UP_ave_6_diff,ED_ave_6_diff,CM_ave_6_diff,MI5_ave_6_diff,One.Percenters_ave_6_diff,BO_ave_6_diff,K_ave_6_diff,HB_ave_6_diff,D_ave_6_diff,M_ave_6_diff,G_ave_6_diff,B_ave_6_diff,T_ave_6_diff,HO_ave_6_diff,I50_ave_6_diff,CL_ave_6_diff,CG_ave_6_diff,R50_ave_6_diff,FF_ave_6_diff,FA_ave_6_diff,AF_ave_6_diff,SC_ave_6_diff,disposal_efficiency_ave_6_diff,form_over_opposition_6_diff,Adj_elo_ave_margin_ave_6_diff,average_elo_opponents_beaten_6_diff,average_elo_opponents_lost_6_diff,Margin_ave_6_diff,implied_odds_prob,implied_odds_prob_away
1508,15345,1429.168961,1609.933983,12.666667,20.333333,96.000000,6049.666667,72.500000,82.000000,14.666667,-0.500000,-2.166667,40.333333,35.500000,-2.666667,-1.166667,-4.666667,-1.666667,1.166667,33.166667,34.333333,5.166667,-2.000000,0.833333,2.000000,0.333333,-6.000000,2.000000,-4.000000,1.833333,2.166667,-3.833333,96.166667,9.333333,0.027756,-2,-18.551481,-175.247348,-5.706047,-15.666667,0.125000,0.877193
1509,15346,1498.995146,1466.844032,12.000000,24.500000,85.000000,5480.166667,71.833333,73.333333,10.666667,-1.000000,-7.666667,61.500000,56.333333,-1.833333,0.833333,5.833333,4.500000,27.000000,29.500000,56.500000,23.000000,-1.500000,2.000000,-2.500000,-2.666667,2.500000,-0.666667,0.333333,2.000000,-1.500000,2.166667,181.333333,77.000000,0.038856,2,-1.689593,-84.083597,132.607241,1.166667,0.628931,0.378788
1510,15347,1668.777582,1287.580776,9.833333,25.333333,68.000000,5577.333333,75.000000,74.833333,12.333333,3.666667,-0.666667,43.166667,56.500000,1.666667,2.666667,-9.166667,0.833333,14.166667,30.333333,44.500000,14.000000,4.500000,-2.500000,-15.500000,-20.500000,3.500000,-1.500000,-2.333333,-0.333333,-3.666667,2.166667,77.000000,202.000000,0.069186,6,46.511420,134.704712,80.111476,42.833333,0.943396,0.055556
1511,15348,1501.969755,1443.532609,10.666667,22.166667,82.833333,5359.166667,72.666667,69.833333,11.166667,2.500000,13.333333,17.500000,25.500000,4.666667,2.333333,-2.500000,9.500000,24.333333,14.000000,38.333333,12.000000,2.333333,2.666667,-1.833333,-4.000000,4.000000,5.833333,3.000000,2.166667,-1.166667,3.500000,130.666667,132.666667,-0.006370,-4,30.663830,124.401787,-72.362236,27.500000,0.806452,0.188679
1512,15349,1285.782360,1633.965757,10.666667,25.666667,81.500000,5663.666667,70.833333,75.166667,11.666667,3.333333,-11.833333,27.000000,26.833333,-1.333333,4.000000,13.666667,-1.000000,-4.333333,18.333333,14.000000,11.666667,1.333333,-1.000000,-9.500000,8.500000,2.333333,1.166667,6.166667,4.666667,3.333333,2.833333,31.000000,61.666667,0.045900,-6,14.897485,-40.126494,32.964402,12.000000,0.423729,0.571429
1513,15350,1633.190328,1500.301134,15.833333,24.666667,123.833333,6241.500000,70.833333,76.500000,16.166667,-4.166667,-18.500000,-26.000000,-38.000000,-1.666667,-6.166667,1.500000,0.833333,-18.833333,-26.000000,-44.833333,-12.166667,-7.166667,-2.333333,-2.666667,-13.833333,-11.500000,-3.666667,-0.833333,8.333333,1.333333,0.000000,-213.500000,-183.833333,-0.017498,4,-58.400243,-6.317653,-46.119168,-58.666667,0.613497,0.387597
1514,15351,1243.047837,1580.237667,11.166667,23.666667,101.500000,5650.166667,69.500000,70.833333,11.833333,-3.500000,-12.000000,-23.166667,-35.166667,-0.333333,-5.000000,-8.833333,-1.666667,-25.833333,-10.000000,-35.833333,-7.166667,-5.166667,-2.166667,-15.166667,-9.333333,-12.000000,-3.333333,4.000000,6.333333,-1.833333,3.333333,-234.000000,-224.333333,-0.022622,-4,-65.692926,58.441510,30.407970,-61.333333,0.142857,0.854701
1515,15352,1601.537522,1452.042224,11.333333,21.833333,75.666667,5388.500000,74.333333,73.333333,11.166667,1.833333,-2.500000,-33.833333,-28.166667,2.666667,1.666667,-2.166667,-3.000000,17.500000,-52.166667,-34.666667,11.333333,1.000000,2.000000,4.000000,16.666667,-1.166667,1.333333,0.333333,0.666667,2.833333,-0.166667,26.166667,89.000000,-0.005179,2,42.936094,91.058792,79.950084,41.500000,0.8849

Great! We now have our feature DataFrame for this weekend's games. Let's now model this to create our predictions

## Creating Our Predictions
To create our predictions we will use similar code to what we used in our AFL Modelling Tutorial. We will need to train our model on all the data we have (up until last week's games), and then use our trained model to predict this week. Again, we will use Stacking with XGB, like we did in our AFL Modelling Tutorial.

In [120]:
afl = prepare_afl_features(window=6, k_factor=24).dropna().sort_values(by='Game')
afl = afl_modelling.get_df_on_one_line(afl)

# Drop columns which leak data and which we don't need
dropped_cols = ['Behinds', 'Goals', 'Opposition Behinds', 'Opposition Goals', 'Opposition Points', 'Points',
               'elo', 'elo_Opp', 'CCL_ave_6', 'SCL_ave_6', 'SI_ave_6', 'MG_ave_6', 'TO_ave_6', 'ITC_ave_6', 'T5_ave_6', 'home_win_away',
               'home_elo_away', 'away_elo_away']
afl = afl.drop(columns=dropped_cols)
# Get a differential DataFrame - subtracting the away features from the home features
diff_df = afl_modelling.get_diff_df(afl)

In [121]:
# Hard code estimators from our last tutorial
all_estimators = [
    LogisticRegression(C=0.75, class_weight=None, dual=False, fit_intercept=True,
       intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
       penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
       verbose=0, warm_start=False),
    
    RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
     max_iter=None, normalize=False, random_state=None, solver='auto',
     tol=0.001),
    
    RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
         max_depth=15, max_features='sqrt', max_leaf_nodes=None,
         min_impurity_decrease=0.0, min_impurity_split=None,
         min_samples_leaf=1, min_samples_split=10,
         min_weight_fraction_leaf=0.0, n_estimators=750, n_jobs=-1,
         oob_score=False, random_state=5, verbose=0, warm_start=False),
    
    GaussianNB(priors=None),
    
    LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
           solver='svd', store_covariance=False, tol=0.0001)
]

# Hard code best cols
best_cols = ['home_elo',
'away_elo',
'GA_ave_6_diff',
'CP_ave_6_diff',
'UP_ave_6_diff',
'ED_ave_6_diff',
'CM_ave_6_diff',
'MI5_ave_6_diff',
'One.Percenters_ave_6_diff',
'BO_ave_6_diff',
'HB_ave_6_diff',
'M_ave_6_diff',
'G_ave_6_diff',
'T_ave_6_diff',
'HO_ave_6_diff',
'I50_ave_6_diff',
'CL_ave_6_diff',
'CG_ave_6_diff',
'R50_ave_6_diff',
'FF_ave_6_diff',
'FA_ave_6_diff',
'AF_ave_6_diff',
'SC_ave_6_diff',
'disposal_efficiency_ave_6_diff',
'R50_efficiency_ave_6_diff',
'I50_efficiency_ave_6_diff',
'Adj_elo_ave_margin_ave_6_diff',
'average_elo_opponents_beaten_6_diff',
'average_elo_opponents_lost_6_diff',
'Margin_ave_6_diff',
'implied_odds_prob',
'implied_odds_prob_away']

In [122]:
# Create our train sets
X = diff_df.drop(columns=['home_win']).select_dtypes(include=[np.number])
y = diff_df['home_win']

features = prediction_feature_set.columns

# Predict Next Round
preds_next_round = afl_modelling.implement_xgb_stacking(X[features].drop(columns='Game'), y, 
                                                        prediction_feature_set.drop(columns='Game'), all_estimators)

C:\Users\wardj\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Now that we have our predictions, the final step is to put our predictions into a DataFrame so they're easy on the eye, and our predictions correspond to actual home and away teams rather than Game IDs. This is the final step.

In [123]:
preds_df = pd.DataFrame({
    "Game": prediction_feature_set['Game'],
    "Prediction (home_win)": preds_next_round
})

prediction_feature_set['home_odds'] = 1 / prediction_feature_set['implied_odds_prob']
prediction_feature_set['away_odds'] = 1 / prediction_feature_set['implied_odds_prob_away']

final_preds_df = pd.merge(preds_df, next_week_df[['home_team', 'away_team', 'Game']], on='Game').drop_duplicates()
final_preds_df = pd.merge(final_preds_df, prediction_feature_set[['home_odds', 'away_odds', 'home_elo', 'away_elo', 'Game']], on='Game')
final_preds_df['Predicted Winner'] = final_preds_df.apply(lambda x: x['home_team'] if x['Prediction (home_win)'] == 1 else x['away_team'], axis=1)

In [125]:
final_preds_df[['Game', 'home_team', 'away_team', 'Predicted Winner', 'home_odds', 'away_odds', 'home_elo', 'away_elo']] 

,Game,home_team,away_team,Predicted Winner,home_odds,away_odds,home_elo,away_elo
0,15345,St Kilda,Richmond,Richmond,8.00,1.14,1429.168961,1609.933983
1,15346,Collingwood,North Melbourne,Collingwood,1.59,2.64,1498.995146,1466.844032
2,15347,Sydney,Gold Coast,Sydney,1.06,18.00,1668.777582,1287.580776
3,15348,Essendon,Fremantle,Essendon,1.24,5.30,1501.969755,1443.532609
4,15349,Brisbane,Adelaide,Brisbane,2.36,1.75,1285.782360,1633.965757
5,15350,Geelong,Melbourne,Melbourne,1.63,2.58,1633.190328,1500.301134
6,15351,Carlton,Hawthorn,Hawthorn,7.00,1.17,1243.047837,1580.237667
7,15352,West Coast,Western Bulldogs,West Coast,1.13,9.40,1601.537522,1452.042224
8,15353,Port Adelaide,GWS,Port Adelaide,1.90,2.10,1566.616129,1596.476198


## Conclusion
Congratulations! You have created AFL predictions for this week. If you are beginner to this, don't be overwhelmed. The process gets easier each time you do it. And it is super rewarding. In future iterations we will update this tutorial to predict actual odds, and then integrate this with Betfair's API so that you can create an automated betting strategy using Machine Learning to create your predictions!